In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [ ]:
##Click on double arrows to begin: Remember, this program works best in Explorer or Firefox

In [ ]:
import matplotlib 
import ipywidgets as widgets
import pandas as pd
import io
import seaborn as sns
import openpyxl
from IPython.display import display, clear_output
from ipywidgets import interact, interactive, fixed, interact_manual, GridspecLayout, Button, Layout,Label, HTML, HBox, Image, VBox, Box
from matplotlib_venn import venn2, venn3
from matplotlib import pyplot as plt
from threading import Timer
from traitlets import traitlets
from ipyevents import Event
import plotly.express as px

In [ ]:
mouse_means = pd.read_csv('https://idk-etl-prod-download-bucket.s3.amazonaws.com/aibs_mouse_ctx-hip_smart-seq/trimmed_means.csv')
human_means = pd.read_csv('https://idk-etl-prod-download-bucket.s3.amazonaws.com/aibs_human_ctx_smart-seq/trimmed_means.csv')

# Get all gene and cell type names: HUMAN
human_idx = list(human_means.columns.values)
human_listOfColumnNames = list(human_means['cluster_label'])
# Get all gene and cell type: MOUSE
mouse_idx = list(mouse_means.columns.values)
mouse_listOfColumnNames = list(mouse_means['cluster_label'])


In [ ]:

mouse_means2 = mouse_means.set_index('cluster_label').T.rename_axis('GeneID').rename_axis(None, axis = 1)
human_means2 = human_means.set_index('cluster_label').T.rename_axis('GeneID').rename_axis(None, axis = 1)


In [ ]:

##Create a class of buttons which allows me to check which one was pressed
class LoadedButton(widgets.Button):
    """A button that can hold a value as a attribute."""

    def __init__(self, value=None, *args, **kwargs):
        super(LoadedButton, self).__init__(*args, **kwargs)
        # Create the value attribute.
        self.add_traits(value=traitlets.Any(value))


button_layout = widgets.Layout(border='solid 1px black', height='auto', width='auto')
def create_expanded_button(description, button_style):
    return LoadedButton(description=description, button_style=button_style, layout=button_layout)


In [ ]:
#define output container to print results into 
out = widgets.Output(layout={'border': '1px solid black'})

##functions needed
def choose_mouse(msCells):
    msCells.button_style = 'success'
    huCells.button_style = ''
    msCells.value = "mouse"
    genes1.options = mouse_idx
    genes2.options = mouse_idx
    genes3.options = mouse_idx   
    
def choose_human(huCells):
    huCells.button_style = 'success'
    msCells.button_style = ''
    huCells.value = "human"
    genes1.options = human_idx
    genes2.options = human_idx
    genes3.options = human_idx
    
    
def on_go_button_clicked(go_button):
    excelname = excel_label.value + ".xlsx"
    genes = [genes1.value, genes2.value, genes3.value]
    #cut list to two if two or fewer selected
    genes_b = list(filter(None, genes))
    if len(genes_b) < 2:
        display ("Please pick at least two genes")
    else:
        venngraph(genes_b, excelname)
        tab.selected_index = 1    

        
def on_reset_button_clicked(reset_button):  
    clear_output()
    genes = []
    grid

In [ ]:
%matplotlib notebook
def venngraph(genes, excelname):

        #out2 = widgets.Output(layout={'border': '1px solid black'})
        venny1, venny2, venny3 = [],[],[]

        if msCells.value == 'mouse':
            venny = mouse_means2.loc[genes]
        if huCells.value == 'human':
            venny = human_means2.loc[genes]

        venny1 = venny.loc[genes[0]]
        venny1 = venny1[venny1 >= 0.5].index.to_list()

        venny2 = venny.loc[genes[1]]
        venny2 = venny2[venny2 >= 0.5].index.to_list()

        v1only = list(set(venny1) - set(venny2))
        v2only = list(set(venny2) - set(venny1))
        overlap_ab = [value for value in venny1 if value in venny2] 
        
        name1 = str(genes[0]) + " only"
        name2 = str(genes[1]) + " only"
        nameab = str(genes[0]) + " and " + str(genes[1])
        
        
        if len(genes) == 2:
            Title = (genes[0] + " / " + genes[1])
            if (len(venny1) >= 1) & (len(venny2) >= 1):
                la,lb,lc = len(v1only),len(v2only),len(overlap_ab)
                # now find the max
                max_len = max(la,lb,lc)
                if not max_len == la:
                    v1only.extend(['']*(max_len-la))
                if not max_len == lb:
                    v2only.extend(['']*(max_len-lb))
                if not max_len == lc:
                    overlap_ab.extend(['']*(max_len-lc))
                    
                venn_df = pd.DataFrame({name1:v1only,name2:v2only,nameab:overlap_ab}) 
                writer = pd.ExcelWriter(excelname, engine='openpyxl')
                venn_df.to_excel(writer, sheet_name='Sheet1')
                writer.save()
                
                with out2:
                    clear_output()
                    plt.figure(figsize=(5, 5), facecolor="white")
                    v = venn2([set(venny1), set(venny2)], set_labels = (genes[0], genes[1]))
                    plt.title(Title)
                    plt.show()
                                
            else:
                print("No Overlap: At least one gene is not expressed over a value of 0.5 in any cell types:", 
                      genes[0], ": expressed in ", len(venny1), "cells; ", genes[1], ": expressed in ", len(venny2), "cells. ")
                
    
        elif len(genes) == 3:
            Title = (genes[0] + " / " + genes[1]+ " / " + genes[2])
            
            if msCells.value == 'mouse':
                venny3 = mouse_means2.loc[genes[2]]
            if huCells.value == 'human':
                venny3 = human_means2.loc[genes[2]]            

            venny3 = venny3[venny3 >= 0.5].index.to_list()
            if (len(venny1) >= 1) & (len(venny2) >= 1) & (len(venny3) >= 1):
                name3 = str(genes[2]) + " only"
                namebc = str(genes[1]) + " and " + str(genes[2])
                nameac = str(genes[0]) + " and " + str(genes[2])
                nameabc = str(genes[0]) + " and " + str(genes[1])+ " and " + str(genes[2])         
                v3only = list(set(venny1) - set(venny2) - set(venny3))
                overlap_bc = [value for value in venny2 if value in venny3]
                overlap_ac = [value for value in venny1 if value in venny3]
                overlap_abc = [value for value in venny3 if value in overlap_ab]

                la,lb,lc, ld, le, lf, lg  = len(v1only),len(v2only),len(v3only),len(overlap_ab),len(overlap_bc),len(overlap_ac),len(overlap_abc)
                # now find the max
                max_len = max(la,lb,lc,ld, le, lf, lg)
                if not max_len == la:
                    v1only.extend(['']*(max_len-la))
                if not max_len == lb:
                    v2only.extend(['']*(max_len-lb))
                if not max_len == lc:
                    v3only.extend(['']*(max_len-lc))
                if not max_len == ld:
                    overlap_ab.extend(['']*(max_len-ld))
                if not max_len == le:
                    overlap_bc.extend(['']*(max_len-le))
                if not max_len == lf:
                    overlap_ac.extend(['']*(max_len-lf))
                if not max_len == lg:
                    overlap_abc.extend(['']*(max_len-lg))

                venn_df = pd.DataFrame({name1:v1only,name2:v2only,name3:v3only,nameab:overlap_ab,namebc:overlap_bc,
                                                nameac:overlap_ac, nameabc: overlap_abc}) 
                writer = pd.ExcelWriter(excelname, engine='openpyxl')
                venn_df.to_excel(writer, sheet_name='Sheet1')
                writer.save() 
                
                with out2:
                    clear_output()
                    plt.figure(figsize=(5, 5), facecolor="white")
                    v = venn3([set(venny1), set(venny2), set(venny3)], set_labels = (genes[0], genes[1], genes[2]))
                    plt.title(Title)
                    plt.show()
                        
            else:
                print("No Overlap: At least one gene is not expressed over a value of 0.5 in any cell types:", 
                      genes[0], ": expressed in ", len(venny1), "cells; ", genes[1], ": expressed in ", len(venny2), "cells; ",
                     genes[2], ": expressed in ", len(venny3), "cells.")

        

In [ ]:
## all the labels
spacer = widgets.Label('')
genetitle = widgets.Label('Select Genes')
choicelabel = widgets.Label('Analyze species:')
excel_label = widgets.Text(value='Name',placeholder='Type something',description='Name Excel:')


    
## The buttons/ widgets and what they do
genes1 =  widgets.Combobox(options= ["Choose a Species"],description='Gene',placeholder='Type here',layout={'width': '80%'})
genes1.observe(choose_human, 'change')
genes1.observe(choose_mouse, 'change')

genes2 =  widgets.Combobox(options= ["Choose a Species"],description='Gene',placeholder='Type here',layout={'width': '80%'})
genes2.observe(choose_human, 'change')
genes2.observe(choose_mouse, 'change')

genes3 =  widgets.Combobox(options= ["Choose a Species"],description='Gene',placeholder='Type here',layout={'width': '80%'})
genes3.observe(choose_human, 'change')
genes3.observe(choose_mouse, 'change')


msCells = LoadedButton(description = "Mouse")
msCells.on_click(choose_mouse)

huCells = LoadedButton(description = "Human")
huCells.on_click(choose_human)

go_button = create_expanded_button('Create Venn', '')
go_button.observe(huCells, 'value')
go_button.observe(genes1, 'value')  
go_button.observe(genes2, 'value')  
go_button.observe(genes3, 'value')  
go_button.on_click(on_go_button_clicked)

reset_button = create_expanded_button('Reset Options', '')
reset_button.on_click(on_reset_button_clicked)


In [ ]:
choicemouse = widgets.VBox([choicelabel,msCells])
speciesbox = widgets.VBox([choicemouse, huCells])
geneshbox = widgets.VBox([genes1, genes2, genes3])
genesbox = widgets.VBox([genetitle,geneshbox])
buttons = widgets.VBox([go_button, reset_button])
excelandgo = widgets.VBox([excel_label, buttons])


box_layout = widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px',
        height='auto', width='auto')

grid = GridspecLayout(1, 3)
grid.layout = box_layout
grid[0,0] = speciesbox
grid[0,1] = genesbox
grid[0, 2:] = excelandgo
grid[0, 0].layout.width = '90%'

In [ ]:
out1 = widgets.Output()
out2 = widgets.Output()


In [ ]:
tab = widgets.Tab(children = [out1, out2])
tab.set_title(0, 'App')
tab.set_title(1, 'Venn Diagram')
tab.selected_index = 0
display(tab)
with out1:
    clear_output()
    display(grid)

with out2:
    clear_output()



